In [15]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
TRAIN = "/home/jungmo/Documents/autonomous/caffe/bird_image/train/train.txt"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_200.txt"
BIRD_ROOT = "/home/jungmo/Documents/autonomous/caffe/"

BINARYPROTO_PATH = os.path.join(BIRD_ROOT, "db/mean.binaryproto")
NPY_PATH = os.path.join(BIRD_ROOT, "db/mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

problem = open('egg_classified_as_bird.txt', "w")
def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "color16/color16_model.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [16]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_200.txt"
evaluate_model("test_16_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_200.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.92 
Confusion Matrix:
[[2448    6   13   33]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.06 
Confusion Matrix:
[[4903   14   22   61]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.39 
Confusion Matrix:
[[6611   19   26   82]
 [  17  693   50    2]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.13 
Confusion Matrix:
[[8403   19   38   92]
 [  39 1310   92    7]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.34 
Confusion Matrix:
[[10818    23    67   100]
 [   39  1310    92     7]
 [    0     0    39     5]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 96.41 
Confusion Matrix:
[[10818    23    67   100]
 [   39  1310    92     7]
 [   

In [17]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_150.txt"
evaluate_model("test_16_150.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_150.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.68 
Confusion Matrix:
[[2467    7    4   22]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.48 
Confusion Matrix:
[[4924   11   13   52]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.47 
Confusion Matrix:
[[6622   20   20   76]
 [  23  688   46    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.28 
Confusion Matrix:
[[8420   22   27   83]
 [  43 1308   89    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.56 
Confusion Matrix:
[[10849    24    34   101]
 [   43  1308    89     8]
 [    1     0    38     5]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 95.13 
Confusion Matrix:
[[10849    24    34   101]
 [   43  1308    89     8]
 [   60    10  2113 

In [18]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_140.txt"
evaluate_model("test_16_140.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_140.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.36 
Confusion Matrix:
[[2459    9    8   24]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.32 
Confusion Matrix:
[[4916   17   14   53]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.52 
Confusion Matrix:
[[6621   21   19   77]
 [  17  693   47    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.22 
Confusion Matrix:
[[8412   24   26   90]
 [  34 1310   96    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.53 
Confusion Matrix:
[[10843    25    36   104]
 [   34  1310    96     8]
 [    1     0    38     5]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.67 
Confusion Matrix:
[[10843    25    36   104]
 [   34  1310    96     8]
 [   64     6  2047 

In [19]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_130.txt"
evaluate_model("test_16_130.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_130.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.88 
Confusion Matrix:
[[2447    7   11   35]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.26 
Confusion Matrix:
[[4913   13   18   56]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.36 
Confusion Matrix:
[[6617   18   21   82]
 [  22  685   51    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.21 
Confusion Matrix:
[[8410   22   24   96]
 [  40 1311   89    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.42 
Confusion Matrix:
[[10838    23    40   107]
 [   40  1311    89     8]
 [    1     0    29    14]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.26 
Confusion Matrix:
[[10838    23    40   107]
 [   40  1311    89     8]
 [   50     6  1990 

In [20]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_120.txt"
evaluate_model("test_16_120.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_120.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.96 
Confusion Matrix:
[[2449    6    9   36]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.10 
Confusion Matrix:
[[4905   11   16   68]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.01 
Confusion Matrix:
[[6602   18   26   92]
 [  27  674   55    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.86 
Confusion Matrix:
[[8389   19   42  102]
 [  43 1297   98   10]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.14 
Confusion Matrix:
[[10807    22    67   112]
 [   43  1297    98    10]
 [    1     0    39     4]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.83 
Confusion Matrix:
[[10807    22    67   112]
 [   43  1297    98    10]
 [   50     7  2120 

In [21]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_110.txt"
evaluate_model("test_16_110.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_110.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.24 
Confusion Matrix:
[[2456    7    8   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.22 
Confusion Matrix:
[[4911   12   15   62]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.27 
Confusion Matrix:
[[6611   18   22   87]
 [  30  684   44    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.00 
Confusion Matrix:
[[8391   19   33  109]
 [  50 1309   78   11]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.20 
Confusion Matrix:
[[10810    22    48   128]
 [   50  1309    78    11]
 [    5     0    31     8]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 92.36 
Confusion Matrix:
[[10810    22    48   128]
 [   50  1309    78    11]
 [  103     6  1735 

In [22]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_100.txt"
evaluate_model("test_16_100.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_16_100.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.16 
Confusion Matrix:
[[2454    7    4   35]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.96 
Confusion Matrix:
[[4898   15   13   74]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.31 
Confusion Matrix:
[[6603   20   13  102]
 [  30  695   29    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.98 
Confusion Matrix:
[[8385   21   17  129]
 [  57 1313   63   15]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.01 
Confusion Matrix:
[[10799    25    19   165]
 [   57  1313    63    15]
 [    7     0    14    23]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 87.61 
Confusion Matrix:
[[10799    25    19   165]
 [   57  1313    63    15]
 [  399     7  1030 